In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import GridSearchCV
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
#from skorch import NeuralNetClassifier
import numpy as np
import os

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
#from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold


In [5]:
train_path='/content/drive/MyDrive/AI project/Training'
test_path='/content/drive/MyDrive/AI project/Testing'


In [41]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_style('whitegrid')
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras import layers,models,optimizers,preprocessing
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout




main_dir = "/content/drive/MyDrive/AI project"
classification_dirs = [("no_tumor",), ("meningioma_tumor",), ("glioma_tumor",), ("pituitary_tumor",)]
resolution = 64

def load_images(root_dir_name):
    x = []
    y = []
    
    for label, sub_dir_names in enumerate(classification_dirs):
        for sub_dir_name in sub_dir_names:
            print(f"loading {root_dir_name} {sub_dir_name}")
            sub_dir_path = os.path.join(main_dir, root_dir_name, sub_dir_name)
            for image_name in os.listdir(sub_dir_path):
                image_path = os.path.join(sub_dir_path, image_name)
                image = preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(resolution, resolution))
                x.append(preprocessing.image.img_to_array(image))
                y.append(label)
    
    x = np.array(x) / 255.0
    y = to_categorical(np.array(y))
    
    return x,y

# Load training and testing images and labels
x_train, y_train = load_images("Training")
x_test, y_test = load_images("Testing")

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Print the shape of the data sets
print("Training set shape:", x_train.shape, y_train.shape)
print("Validation set shape:", x_val.shape, y_val.shape)
print("Testing set shape:", x_test.shape, y_test.shape)


loading Training no_tumor
loading Training meningioma_tumor
loading Training glioma_tumor
loading Training pituitary_tumor
loading Testing no_tumor
loading Testing meningioma_tumor
loading Testing glioma_tumor
loading Testing pituitary_tumor
Training set shape: (2296, 64, 64, 1) (2296, 4)
Validation set shape: (574, 64, 64, 1) (574, 4)
Testing set shape: (394, 64, 64, 1) (394, 4)


In [36]:
!pip install tensorflow-addons


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 41.4 MB/s eta 0:00:00


In [47]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Define input shape
input_shape = (64, 64, 1)
n=4

# Define wrapper function that builds and returns the Keras model
def create_model(input_shape, n, dropout_rate1, dropout_rate2):
    model = models.Sequential()

    model.add(Conv2D(32, kernel_size=(2, 2), strides=(1, 1), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(64, kernel_size=(2, 2), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(128, kernel_size=(2, 2), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())

    model.add(Dense(units=1024, activation='relu'))
    model.add(Dropout(dropout_rate1))

    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(dropout_rate2))

    model.add(Dense(units=n, activation='softmax'))

    optimizer = optimizers.Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [42]:
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# # Define input shape
# input_shape = (64, 64, 1)
# n=4

# # Define wrapper function that builds and returns the Keras model
# def create_model(input_shape, n):
#     model = models.Sequential()

#     model.add(Conv2D(32, kernel_size=(2, 2), strides=(1, 1), activation='relu', input_shape=input_shape))
#     model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#     model.add(Conv2D(64, kernel_size=(2, 2), strides=(1, 1), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#     model.add(Conv2D(128, kernel_size=(2, 2), strides=(1, 1), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
#     model.add(Flatten())

#     model.add(Dense(units=1024, activation='relu'))
#     model.add(Dropout(0.5))

#     model.add(Dense(units=512, activation='relu'))
#     model.add(Dropout(0.5))

#     model.add(Dense(units=n, activation='softmax'))

#     optimizer = optimizers.Adam(lr=0.0001)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     return model



In [48]:


from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Load training and testing images and labels
x_train, y_train = load_images("Training")
x_test, y_test = load_images("Testing")

# Split training set into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Create a KerasClassifier object
num_classes = 4  # Example value
model = KerasClassifier(build_fn=create_model, input_shape=input_shape, n=n)


# Define hyperparameters to tune
batch_size = [32, 64]
epochs = [5, 10]
dropout_rate1 = [0.2, 0.5]
dropout_rate2 = [0.2, 0.5]
param_grid = dict(batch_size=batch_size, epochs=epochs,dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2)

# Create a random search object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2)

# Fit the random search object to the data
random_search_results = random_search.fit(x_train, y_train, validation_data=(x_val, y_val))

# Print the best parameters and accuracy
print("Best parameters: ", random_search_results.best_params_)
print("Best accuracy: ", random_search_results.best_score_)

# Evaluate the model on the test set
loss, accuracy = random_search_results.best_estimator_.model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Evaluate the best model on the validation set
best_model = random_search_results.best_estimator_.model
val_loss, val_accuracy = best_model.evaluate(x_val, y_val, verbose=0)
print(f"Validation loss: {val_loss:.4f}")
print(f"Validation accuracy: {val_accuracy:.4f}")



loading Training no_tumor
loading Training meningioma_tumor
loading Training glioma_tumor
loading Training pituitary_tumor
loading Testing no_tumor
loading Testing meningioma_tumor
loading Testing glioma_tumor
loading Testing pituitary_tumor


<ipython-input-48-a19e330465f1>:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, input_shape=input_shape, n=n)


Epoch 1/5
36/36 [==============================] - 12s 270ms/step - loss: 1.1996 - accuracy: 0.4660 - val_loss: 0.9883 - val_accuracy: 0.6010
Epoch 2/5
36/36 [==============================] - 11s 298ms/step - loss: 0.8525 - accuracy: 0.6368 - val_loss: 0.9234 - val_accuracy: 0.6167
Epoch 3/5
36/36 [==============================] - 11s 308ms/step - loss: 0.6808 - accuracy: 0.7169 - val_loss: 0.6850 - val_accuracy: 0.7178
Epoch 4/5
36/36 [==============================] - 9s 243ms/step - loss: 0.5399 - accuracy: 0.7674 - val_loss: 0.5918 - val_accuracy: 0.7735
Epoch 5/5
36/36 [==============================] - 3s 68ms/step - loss: 0.7598 - accuracy: 0.7195


Epoch 1/5
36/36 [==============================] - 12s 288ms/step - loss: 1.2058 - accuracy: 0.4730 - val_loss: 1.0373 - val_accuracy: 0.5017
Epoch 2/5
36/36 [==============================] - 9s 253ms/step - loss: 0.9428 - accuracy: 0.5958 - val_loss: 0.8142 - val_accuracy: 0.6202
Epoch 3/5
36/36 [==============================] - 10s 284ms/step - loss: 0.7394 - accuracy: 0.6699 - val_loss: 0.6816 - val_accuracy: 0.6986
Epoch 4/5
36/36 [==============================] - 11s 308ms/step - loss: 0.5793 - accuracy: 0.7631 - val_loss: 0.6005 - val_accuracy: 0.7352
Epoch 5/5
36/36 [==============================] - 3s 76ms/step - loss: 0.5518 - accuracy: 0.7875


Epoch 1/10
36/36 [==============================] - 11s 250ms/step - loss: 1.1564 - accuracy: 0.4660 - val_loss: 0.9044 - val_accuracy: 0.6638
Epoch 2/10
36/36 [==============================] - 11s 298ms/step - loss: 0.8533 - accuracy: 0.6350 - val_loss: 0.7842 - val_accuracy: 0.6899
Epoch 3/10
36/36 [==============================] - 11s 294ms/step - loss: 0.6409 - accuracy: 0.7282 - val_loss: 0.5939 - val_accuracy: 0.7491
Epoch 4/10
36/36 [==============================] - 9s 243ms/step - loss: 0.5096 - accuracy: 0.7831 - val_loss: 0.5751 - val_accuracy: 0.7700
Epoch 5/10
36/36 [==============================] - 10s 284ms/step - loss: 0.4111 - accuracy: 0.8423 - val_loss: 0.5534 - val_accuracy: 0.7857
Epoch 6/10
36/36 [==============================] - 11s 298ms/step - loss: 0.3159 - accuracy: 0.8841 - val_loss: 0.4768 - val_accuracy: 0.8171
Epoch 7/10
36/36 [==============================] - 9s 243ms/step - loss: 0.3060 - accuracy: 0.8850 - val_loss: 0.5242 - val_accuracy: 0.8101
E

...

Epoch 1/5
18/18 [==============================] - 11s 546ms/step - loss: 1.2165 - accuracy: 0.4338 - val_loss: 1.0591 - val_accuracy: 0.5052
Epoch 2/5
18/18 [==============================] - 9s 506ms/step - loss: 0.9531 - accuracy: 0.5810 - val_loss: 0.8485 - val_accuracy: 0.6010
Epoch 3/5
18/18 [==============================] - 7s 383ms/step - loss: 0.7570 - accuracy: 0.6733 - val_loss: 0.7062 - val_accuracy: 0.6812
Epoch 4/5
18/18 [==============================] - 9s 538ms/step - loss: 0.6283 - accuracy: 0.7448 - val_loss: 0.6497 - val_accuracy: 0.7195
Epoch 5/5
18/18 [==============================] - 3s 148ms/step - loss: 0.6014 - accuracy: 0.7491


Epoch 1/5
18/18 [==============================] - 9s 434ms/step - loss: 1.2918 - accuracy: 0.3955 - val_loss: 1.1244 - val_accuracy: 0.4739
Epoch 2/5
18/18 [==============================] - 9s 489ms/step - loss: 1.0282 - accuracy: 0.5653 - val_loss: 0.9170 - val_accuracy: 0.5993
Epoch 3/5
18/18 [==============================] - 9s 507ms/step - loss: 0.8192 - accuracy: 0.6490 - val_loss: 0.7966 - val_accuracy: 0.6359
Epoch 4/5
18/18 [==============================] - 7s 390ms/step - loss: 0.7160 - accuracy: 0.6890 - val_loss: 0.6776 - val_accuracy: 0.6847
Epoch 5/5
18/18 [==============================] - 2s 85ms/step - loss: 0.6388 - accuracy: 0.7221


Epoch 1/5
36/36 [==============================] - 12s 302ms/step - loss: 1.1975 - accuracy: 0.4704 - val_loss: 1.0811 - val_accuracy: 0.5557
Epoch 2/5
36/36 [==============================] - 10s 279ms/step - loss: 0.9106 - accuracy: 0.6028 - val_loss: 0.7400 - val_accuracy: 0.6551
Epoch 3/5
36/36 [==============================] - 9s 250ms/step - loss: 0.6886 - accuracy: 0.7091 - val_loss: 0.6499 - val_accuracy: 0.6916
Epoch 4/5
36/36 [==============================] - 11s 297ms/step - loss: 0.5605 - accuracy: 0.7779 - val_loss: 0.5904 - val_accuracy: 0.7596
Epoch 5/5
36/36 [==============================] - 2s 47ms/step - loss: 0.5229 - accuracy: 0.7875


Epoch 1/5
36/36 [==============================] - 12s 304ms/step - loss: 1.2142 - accuracy: 0.4625 - val_loss: 0.9821 - val_accuracy: 0.5784
Epoch 2/5
36/36 [==============================] - 11s 300ms/step - loss: 0.8546 - accuracy: 0.6437 - val_loss: 0.6977 - val_accuracy: 0.6847
Epoch 3/5
36/36 [==============================] - 8s 234ms/step - loss: 0.7137 - accuracy: 0.6838 - val_loss: 0.6895 - val_accuracy: 0.6760
Epoch 4/5
36/36 [==============================] - 11s 291ms/step - loss: 0.5530 - accuracy: 0.7561 - val_loss: 0.5397 - val_accuracy: 0.7648
Epoch 5/5
36/36 [==============================] - 2s 48ms/step - loss: 0.5146 - accuracy: 0.8014


Epoch 1/10
72/72 [==============================] - 19s 254ms/step - loss: 1.0031 - accuracy: 0.5623 - val_loss: 0.7625 - val_accuracy: 0.6603
Epoch 2/10
72/72 [==============================] - 20s 272ms/step - loss: 0.6124 - accuracy: 0.7439 - val_loss: 0.5128 - val_accuracy: 0.7927
Epoch 3/10
72/72 [==============================] - 18s 251ms/step - loss: 0.4550 - accuracy: 0.8166 - val_loss: 0.4051 - val_accuracy: 0.8432
Epoch 4/10
72/72 [==============================] - 18s 252ms/step - loss: 0.3223 - accuracy: 0.8776 - val_loss: 0.3298 - val_accuracy: 0.8659
Epoch 5/10
72/72 [==============================] - 19s 272ms/step - loss: 0.2272 - accuracy: 0.9103 - val_loss: 0.3987 - val_accuracy: 0.8606
Epoch 6/10
72/72 [==============================] - 18s 249ms/step - loss: 0.1919 - accuracy: 0.9247 - val_loss: 0.4524 - val_accuracy: 0.8240
Epoch 7/10
72/72 [==============================] - 18s 253ms/step - loss: 0.1326 - accuracy: 0.9543 - val_loss: 0.3301 - val_accuracy: 0.8885